In [ ]:
# event_collector.py
import numpy as np
import time
import requests
import pandas as pd
import logging
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuração de Logging
logging.basicConfig(level=logger.info, format='%(asctime)s - %(levelname)s - %(message)s')

# Intervalo de datas
start_date = datetime(2025, 1, 17)
end_date = datetime(2025, 9, 4)

# Detalhes da API
url_events = "https://api.b365api.com/v3/events/ended"
token = "183604-pWN7flhoAsWGu8"

def make_request_with_retry(url, params, retries=5, backoff_factor=2, timeout=10):
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            wait_time = backoff_factor ** attempt
            logger.warning(f"Tentativa {attempt} falhou: {e}. Retentando em {wait_time} segundos.")
            time.sleep(wait_time)
            if attempt == retries:
                logger.error(f"Falha após {retries} tentativas para URL: {url}")
                raise e

def fetch_events_for_date(date):
    formatted_date = date.strftime('%Y%m%d')
    params = {
        "token": token,
        "sport_id": "1",
        "league_id": "22614",
        "day": formatted_date,
        "page": 1
    }
    events = []
    while True:
        try:
            response = make_request_with_retry(url_events, params)
            data = response.json()
            events.extend(data['results'])
            if params['page'] * data['pager']['per_page'] < data['pager']['total']:
                params['page'] += 1
            else:
                break
        except Exception as e:
            logger.error(f"Erro ao buscar eventos para {formatted_date}: {e}")
            break
    return events

# Coletar eventos
all_events = []
for single_date in pd.date_range(start_date, end_date):
    events = fetch_events_for_date(single_date)
    all_events.extend(events)
    logger.info(f"Eventos coletados em {single_date.strftime('%Y-%m-%d')}: {len(events)}")

# Criar DataFrame
df_events = pd.DataFrame([{
    'event_id': event['id'],
    'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),
    'time_fora': event['away']['name'].split('(')[0].strip(),
    'jogador_fora': event['away']['name'].split('(')[1].split(')')[0] if '(' in event['away']['name'] else '',
    'gols_fora': event.get('scores', {}).get('2', {}).get('away', 'N/A'),
    'time_casa': event['home']['name'].split('(')[0].strip(),
    'jogador_casa': event['home']['name'].split('(')[1].split(')')[0] if '(' in event['home']['name'] else '',
    'gols_casa': event.get('scores', {}).get('2', {}).get('home', 'N/A'),
} for event in all_events])

# Processar colunas numéricas
df_events['gols_casa'] = pd.to_numeric(df_events['gols_casa'].replace('N/A', np.nan), errors='coerce')
df_events['gols_fora'] = pd.to_numeric(df_events['gols_fora'].replace('N/A', np.nan), errors='coerce')
df_events['gols_totais'] = df_events['gols_casa'] + df_events['gols_fora']
df_events.sort_values(by='date', inplace=True)

# Salvar dados brutos
df_events.to_csv('dados_eventos.csv', index=False)
df_events.to_excel('dados_eventos.xlsx', index=False)
logger.info("Dados dos eventos salvos com sucesso.")